# 1. Подготовка датасета

На данном этапе производится загрузка и предварительная обработка датасетов для задачи детекции фишинговых email сообщений. Используются два датасета:
- **Nazario Phishing Corpus** (2015-2024) - коллекция фишинговых писем
- **Enron Email Dataset** - коллекция легитимных писем

**Важно**: Парсинг email, извлечение признаков и другие операции выполняются через модули системы (`src/email_parser.py`, `src/feature_extractor.py` и т.д.). В данном блокноте выполняется только загрузка и подготовка сырых данных.




In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import sys
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Добавляем путь к src для импорта модулей
sys.path.append(str(Path('../src').resolve()))

# Настройка путей
BASE_DIR = Path('../')
DATA_RAW = BASE_DIR / 'data' / 'raw'
DATA_PROCESSED = BASE_DIR / 'data' / 'processed'
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)

print(f"Рабочая директория: {BASE_DIR.resolve()}")
print(f"Исходные данные: {DATA_RAW.resolve()}")
print(f"Обработанные данные: {DATA_PROCESSED.resolve()}")


Рабочая директория: C:\Users\huzch\Documents\project\eml-phishing-analyzer
Исходные данные: C:\Users\huzch\Documents\project\eml-phishing-analyzer\data\raw
Обработанные данные: C:\Users\huzch\Documents\project\eml-phishing-analyzer\data\processed


## 2. Загрузка Nazario Phishing Corpus

Nazario Phishing Corpus представляет из себя отдельные файлы с письмами за период 2015-2024. Файлы представлены в текстовом формате, письма начинаются с "From " и разделены пустой строкой.

In [2]:
def split_nazario_file(file_path: Path) -> list:
    """
    Разделяет файл Nazario на отдельные email сообщения.
    
    Args:
        file_path: Путь к файлу
        
    Returns:
        list: Список строк с содержимым каждого email
    """
    emails = []
    current_email = []
    
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
        for line in f:
            # Проверяем, начинается ли строка с "From " (начало нового письма)
            if line.startswith('From ') and len(line) > 20 and '@' in line:
                if current_email:
                    emails.append(''.join(current_email))
                current_email = [line]
            else:
                current_email.append(line)
        
        # Добавляем последнее письмо
        if current_email:
            emails.append(''.join(current_email))
    
    return emails


# Загрузка Nazario Phishing Corpus (2015-2024)
nazario_path = DATA_RAW / 'nazario'
nazario_files = sorted(nazario_path.glob('phishing-*.txt'))

print(f"Найдено файлов Nazario: {len(nazario_files)}")
for f in nazario_files:
    print(f"  - {f.name}")

# Объединение всех файлов
all_nazario_emails = []
for file_path in tqdm(nazario_files, desc="Загрузка Nazario файлов"):
    emails = split_nazario_file(file_path)
    all_nazario_emails.extend(emails)
    print(f"  {file_path.name}: {len(emails)} писем")

print(f"\nВсего загружено фишинговых писем: {len(all_nazario_emails)}")


Найдено файлов Nazario: 10
  - phishing-2015.txt
  - phishing-2016.txt
  - phishing-2017.txt
  - phishing-2018.txt
  - phishing-2019.txt
  - phishing-2020.txt
  - phishing-2021.txt
  - phishing-2022.txt
  - phishing-2023.txt
  - phishing-2024.txt


Загрузка Nazario файлов:   0%|          | 0/10 [00:00<?, ?it/s]

Загрузка Nazario файлов:  10%|█         | 1/10 [00:00<00:01,  5.82it/s]

  phishing-2015.txt: 304 писем


Загрузка Nazario файлов:  30%|███       | 3/10 [00:00<00:01,  5.07it/s]

...

  phishing-2018.txt: 288 писем
  phishing-2019.txt: 242 писем
  phishing-2020.txt: 158 писем


Загрузка Nazario файлов:  80%|████████  | 8/10 [00:01<00:00,  9.78it/s]

  phishing-2021.txt: 101 писем
  phishing-2022.txt: 245 писем
  phishing-2023.txt: 419 писем


Загрузка Nazario файлов: 100%|██████████| 10/10 [00:01<00:00,  6.65it/s]

  phishing-2024.txt: 403 писем

Всего загружено фишинговых писем: 2978


Выведем несколько писем для проверки корректной обработки датасета.

In [3]:
emails_nazario_example = all_nazario_emails[:5]
emails_nazario_example

["From MAILER-DAEMON Thu Sep 28 09:57:25 2017\nDate: 28 Sep 2017 09:57:25 -0400\nFrom: Mail System Internal Data <MAILER-DAEMON@monkey.org>\nSubject: DON'T DELETE THIS MESSAGE -- FOLDER INTERNAL DATA\nMessage-ID: <1506607045@monkey.org>\nX-IMAP: 1506607044 0000000307 $Forwarded\nStatus: RO\n\nThis text is part of the internal format of your mail folder, and is not\na real message.  It is created automatically by the mail system software.\nIf deleted, important folder data will be lost, and it will be re-created\nwith the data reset to initial values.\n\n",
 'From 125117@ihp-osb-lngweb5.ihp.iinet.net.au  Fri Jan  2 07:19:00 2015\nReturn-Path: <125117@ihp-osb-lngweb5.ihp.iinet.net.au>\nX-Original-To: jose@login.monkey.org\nDelivered-To: jose@login.monkey.org\nReceived: from forward.b.hostedemail.com (forward.b.hostedemail.com [64.98.36.17])\n\tby l.monkey.org (Postfix) with ESMTP id 180E6F4007\n\tfor <jose@login.monkey.org>; Fri,  2 Jan 2015 07:19:00 -0600 (CST)\nReceived: from smtpin10.

### 2.1 Сохранение Nazario датасета

После загрузки Nazario датасет сохраняется в структурированном формате (CSV) для дальнейшей обработки.


In [4]:
# Сохранение в DataFrame для дальнейшей обработки
print("Создание DataFrame для Nazario датасета...")
df_nazario = pd.DataFrame({
    'email_content': all_nazario_emails,
    'label': 1 # фишинговое письмо
})


Создание DataFrame для Nazario датасета...


In [5]:
df_nazario.head()

,email_content,label
0,From MAILER-DAEMON Thu Sep 28 09:57:25 2017\nD...,1
1,From 125117@ihp-osb-lngweb5.ihp.iinet.net.au ...,1
2,From 125117@ihp-osb-lngweb5.ihp.iinet.net.au ...,1
3,From kenzel2380@suddenlink.net Sat Jan 3 16:...,1
4,From bjones2010@suddenlink.net Tue Jan 6 15:...,1


In [6]:
print(f"Размер датасета Nazario: {len(df_nazario)}")
print(f"Распределение меток: {df_nazario['label'].value_counts().to_dict()}")

Размер датасета Nazario: 2978
Распределение меток: {1: 2978}


In [7]:
# Сохранение Nazario в CSV
print("\nСохранение Nazario датасета в CSV...")
nazario_csv_path = DATA_PROCESSED / 'nazario_raw.csv'
with tqdm(total=1, desc="Сохранение файла", unit="файл", 
          colour='#A23B72', ncols=100, 
          bar_format='{l_bar}{bar}| {elapsed}<{remaining}') as pbar:
    df_nazario.to_csv(nazario_csv_path, index=False)
    pbar.update(1)

print(f"Nazario датасет сохранен: {nazario_csv_path}")


Сохранение Nazario датасета в CSV...


Сохранение файла: 100%|████████████████████████████████████████████████████████████████| 00:03<00:00

Nazario датасет сохранен: ..\data\processed\nazario_raw.csv


## 3. Загрузка Enron Email Dataset

Enron Email Dataset содержит легитимные письма. Файл представлен в формате CSV. После загрузки будет выполнено сравнение размеров с Nazario датасетом и балансировка при необходимости.


In [8]:
# Определение структуры Enron файла
enron_path = DATA_RAW / 'enron' / 'emails.csv'

# Читаем первые строки для понимания структуры
sample_df = pd.read_csv(enron_path).head()
print(f"Столбцы Enron: {sample_df.columns.tolist()}")
print(f"\nПример данных:")
print(sample_df)

# Определяем столбец с содержимым email
email_content_col = 'message'
print(f"\nИспользуется столбец: {email_content_col}")


Столбцы Enron: ['file', 'message']

Пример данных:
                       file                                            message
0     allen-p/_sent_mail/1.  Message-ID: <18782981.1075855378110.JavaMail.e...
1    allen-p/_sent_mail/10.  Message-ID: <15464986.1075855378456.JavaMail.e...
2   allen-p/_sent_mail/100.  Message-ID: <24216240.1075855687451.JavaMail.e...
3  allen-p/_sent_mail/1000.  Message-ID: <13505866.1075863688222.JavaMail.e...
4  allen-p/_sent_mail/1001.  Message-ID: <30922949.1075863688243.JavaMail.e...

Используется столбец: message


In [9]:
# Загрузка Enron (по частям из-за большого размера)
chunk_size = 10000
enron_data = []

# Загружаем все доступные данные 
for chunk in tqdm(pd.read_csv(enron_path, chunksize=chunk_size), desc="Чтение Enron"):
    # Берем только нужный столбец
    chunk_emails = chunk[[email_content_col]].copy()
    chunk_emails.columns = ['email_content']
    enron_data.append(chunk_emails)

df_enron_raw = pd.concat(enron_data, ignore_index=True)

print(f"\nЗагружено легитимных писем: {len(df_enron_raw)}")


Чтение Enron: 0it [00:00, ?it/s]

Чтение Enron: 52it [00:40,  1.28it/s]


Загружено легитимных писем: 517401


In [10]:
# Уменьшение размера Enron до 2950 строк для балансировки
import random
target_size = 2950
print(f"Уменьшение размера Enron с {len(df_enron_raw)} до {target_size} строк...")
random.seed(42)
df_enron_raw = df_enron_raw.sample(n=target_size, random_state=42).reset_index(drop=True)
print(f"Размер после уменьшения: {len(df_enron_raw)}")


Уменьшение размера Enron с 517401 до 2950 строк...
Размер после уменьшения: 2950


### 3.1 Сохранение Enron датасета

После загрузки Enron датасет сохраняется в структурированном формате (CSV) для дальнейшей обработки.


In [11]:
# Сохранение в DataFrame для дальнейшей обработки
print("Создание DataFrame для Enron датасета...")
df_enron = pd.DataFrame({
    'email_content': df_enron_raw['email_content'],
    'label': 0  # легитимное письмо
})

Создание DataFrame для Enron датасета...


In [12]:
df_enron.head()


,email_content,label
0,Message-ID: <21013688.1075844564560.JavaMail.e...,0
1,Message-ID: <22688499.1075854130303.JavaMail.e...,0
2,Message-ID: <27817771.1075841359502.JavaMail.e...,0
3,Message-ID: <10695160.1075858510449.JavaMail.e...,0
4,Message-ID: <27819143.1075853689038.JavaMail.e...,0


In [13]:
print(f"Размер датасета Enron: {len(df_enron)}")
print(f"Распределение меток: {df_enron['label'].value_counts().to_dict()}")


Размер датасета Enron: 2950
Распределение меток: {0: 2950}


In [14]:
# Сохранение Enron в CSV
print("\nСохранение Enron датасета в CSV...")
enron_csv_path = DATA_PROCESSED / 'enron_raw.csv'
with tqdm(total=1, desc="Сохранение файла", unit="файл", 
          colour='#A23B72', ncols=100, 
          bar_format='{l_bar}{bar}| {elapsed}<{remaining}') as pbar:
    df_enron.to_csv(enron_csv_path, index=False)
    pbar.update(1)

print(f"Enron датасет сохранен: {enron_csv_path}")


Сохранение Enron датасета в CSV...


Сохранение файла: 100%|████████████████████████████████████████████████████████████████| 00:00<00:00

Enron датасет сохранен: ..\data\processed\enron_raw.csv


In [15]:
# Сравнение размеров датасетов
nazario_size = len(df_nazario)
enron_size = len(df_enron)

print(f"Размер Nazario (phishing): {nazario_size}")
print(f"Размер Enron (legitimate): {enron_size}")
print(f"Разница: {abs(nazario_size - enron_size)}")

# Балансировка: привести оба датасета к минимальному размеру
min_size = min(nazario_size, enron_size)
if nazario_size != enron_size:
    print(f"\nБалансировка датасетов до минимального размера ({min_size})...")
    df_nazario = df_nazario.sample(n=min_size, random_state=42).reset_index(drop=True)
    df_enron = df_enron.sample(n=min_size, random_state=42).reset_index(drop=True)
    print(f"Размер Nazario после балансировки: {len(df_nazario)}")
    print(f"Размер Enron после балансировки: {len(df_enron)}")
else:
    print("\nРазмеры датасетов уже сбалансированы.")

print(f"\nФинальные размеры:")
print(f"  Nazario (phishing): {len(df_nazario)}")
print(f"  Enron (legitimate): {len(df_enron)}")


Размер Nazario (phishing): 2978
Размер Enron (legitimate): 2950
Разница: 28

Балансировка датасетов до минимального размера (2950)...
Размер Nazario после балансировки: 2950
Размер Enron после балансировки: 2950

Финальные размеры:
  Nazario (phishing): 2950
  Enron (legitimate): 2950


## 4. Объединение датасетов

Объединение фишинговых и легитимных писем в единый датасет для дальнейшей обработки модулями системы.


In [16]:
# Объединение Nazario и Enron в общий датафрейм
df = pd.concat([df_nazario, df_enron], ignore_index=True)

### 4.1 Проверка и очистка данных

In [17]:
# Проверка пропущенных значений
print("Пропущенные значения:")
print(df.isnull().sum())

# Проверка пустых писем
empty_emails = df['email_content'].str.strip().str.len() == 0
print(f"Пустых писем: {empty_emails.sum()}")
if empty_emails.sum() > 0:
    df = df[~empty_emails]
    print(f"Размерность после удаления пустых: {df.shape}")

print(f"\nФинальное распределение классов:")
print(df['label'].value_counts())

Пропущенные значения:
email_content    0
label            0
dtype: int64
Пустых писем: 0

Финальное распределение классов:
label
1    2950
0    2950
Name: count, dtype: int64


In [18]:
df.head()

,email_content,label
0,From jose@monkey.org Thu Aug 17 11:41:23 2023 ...,1
1,From jose@monkey.org Fri Sep 27 15:46:37 2024 ...,1
2,From jose@monkey.org Mon Apr 20 11:26:14 2020 ...,1
3,From jose@monkey.org Thu Sep 21 19:14:02 2017 ...,1
4,From jose@monkey.org Wed May 3 17:20:00 2017 ...,1


In [19]:
df.tail()

,email_content,label
5895,Message-ID: <17702800.1075856096893.JavaMail.e...,0
5896,Message-ID: <16654602.1075840221753.JavaMail.e...,0
5897,Message-ID: <21859410.1075852282055.JavaMail.e...,0
5898,Message-ID: <11613616.1075849659137.JavaMail.e...,0
5899,Message-ID: <19463108.1075852096010.JavaMail.e...,0


In [20]:
# Сохранение итогового датасета
output_path = DATA_PROCESSED / 'email_dataset.csv'
df.to_csv(output_path, index=False)
print(f"Итоговый датасет сохранён в {output_path}")

Итоговый датасет сохранён в ..\data\processed\email_dataset.csv


In [21]:
# Сохраняем первые 5 и последние 5 строк df в файл dataset_example.csv
example_rows = pd.concat([df.head(5), df.tail(5)])
example_output_path = DATA_PROCESSED / 'dataset_example.csv'
example_rows.to_csv(example_output_path, index=False)
print(f"Первые и последние 5 строк сохранены в {example_output_path}")


Первые и последние 5 строк сохранены в ..\data\processed\dataset_example.csv


## 5. Проверка работоспособности парсинга

Проверка парсинга на первом и последнем письме из итогового датасета для валидации корректности работы модуля `email_parser.py`.


### 5.1 Парсинг первого письма

In [22]:
from email_parser import parse_email
import json
import logging

In [23]:
# Получаем первое письмо из датасета
first_email = df.iloc[1]['email_content']
first_label = df.iloc[1]['label']

In [24]:
# Настройте логирование в начале вашего блокнота/скрипта
logging.basicConfig(
    level=logging.DEBUG,  # или INFO/WARNING/ERROR
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(),  # В консоль
        logging.FileHandler('email_parsing.log')  # В файл
    ]
)

In [25]:
# Функция для парсинга письма по индексу
def parse_and_print_email_by_index(email_df, idx):
    if idx < 0 or idx >= len(email_df):
        print(f"Индекс {idx} вне диапазона (0 ... {len(email_df)-1})")
        return
    
    email_content = email_df.iloc[idx]['email_content']
    label = email_df.iloc[idx]['label']

    print("=" * 80)
    print(f"ПАРСИНГ ПИСЬМА ПО ИНДЕКСУ {idx}")
    print("=" * 80)
    print(f"Индекс: {idx}")
    print(f"Метка (label): {label} ({'Фишинговое' if label == 1 else 'Легитимное'})")
    print(f"Длина исходного текста: {len(email_content)} символов")
    print("\n" + "-" * 80)
    print("ИСХОДНОЕ ПИСЬМО (первые 500 символов):")
    print("-" * 80)
    print(email_content[:500])
    print("...")
    print("\n" + "-" * 80)

    # Парсинг письма
    try:
        parsed_mail = parse_email(email_content)
        
        print("РЕЗУЛЬТАТЫ ПАРСИНГА:")
        print("-" * 80)
        
        # ВСЕ ЗАГОЛОВКИ
        headers = parsed_mail.get('headers', {})
        print(f"\n📧 ВСЕ ЗАГОЛОВКИ ({len(headers)}):")
        print("-" * 80)
        for header_name, header_value in sorted(headers.items()):
            value_str = str(header_value)
            # Ограничиваем длину для очень длинных значений
            if len(value_str) > 200:
                value_str = value_str[:200] + "..."
            print(f"  • {header_name.upper()}: {value_str}")
        
        # ТЕЛО ПИСЬМА (полное)
        body = parsed_mail.get('body', {})
        text_body = body.get('text', '')
        html_body = body.get('html', '')
        text_len = len(text_body)
        html_len = len(html_body)
        
        print(f"\n📝 ТЕЛО ПИСЬМА:")
        print("-" * 80)
        print(f"  • Текст (text/plain): {text_len} символов")
        if text_len > 0:
            print(f"\n  ПОЛНЫЙ ТЕКСТ:")
            print("  " + "-" * 76)
            # Выводим текст с отступом, ограничиваем до 2000 символов для читаемости
            text_preview = text_body[:2000] if text_len > 2000 else text_body
            for line in text_preview.split('\n'):
                print(f"  {line}")
            if text_len > 2000:
                print(f"  ... (еще {text_len - 2000} символов)")
        
        print(f"\n  • HTML (text/html): {html_len} символов")
        if html_len > 0:
            print(f"\n  ПРЕВЬЮ HTML (первые 1000 символов):")
            print("  " + "-" * 76)
            html_preview = html_body[:1000]
            print(f"  {html_preview}")
            if html_len > 1000:
                print(f"  ... (еще {html_len - 1000} символов)")
        
        # ВСЕ URL
        urls = parsed_mail.get('urls', [])
        print(f"\n🔗 ВСЕ URL ({len(urls)}):")
        print("-" * 80)
        if urls:
            for i, url in enumerate(urls, 1):
                print(f"  {i}. {url}")
        else:
            print("  URL не найдены")
        
        # ВСЕ ДОМЕНЫ И IP-АДРЕСА
        domains = parsed_mail.get('domains', {})
        print(f"\n🌐 ВСЕ ДОМЕНЫ И IP-АДРЕСА:")
        print("-" * 80)
        
        all_domains = domains.get('domains', [])
        print(f"  • Домены из URL ({len(all_domains)}):")
        if all_domains:
            for i, domain in enumerate(all_domains, 1):
                print(f"    {i}. {domain}")
        else:
            print("    Домены не найдены")
        
        all_ips = domains.get('ips', [])
        print(f"\n  • IP-адреса ({len(all_ips)}):")
        if all_ips:
            for i, ip in enumerate(all_ips, 1):
                print(f"    {i}. {ip}")
        else:
            print("    IP-адреса не найдены")
        
        email_domains = domains.get('email_domains', [])
        print(f"\n  • Домены из email адресов ({len(email_domains)}):")
        if email_domains:
            for i, domain in enumerate(email_domains, 1):
                print(f"    {i}. {domain}")
        else:
            print("    Домены из email не найдены")
        
        # ВСЕ ВЛОЖЕНИЯ (с полной информацией)
        attachments = parsed_mail.get('attachments_metadata', [])
        attachment_hashes = parsed_mail.get('attachment_hashes', [])
        print(f"\n📎 ВСЕ ВЛОЖЕНИЯ ({len(attachments)}):")
        print("-" * 80)
        if attachments:
            for i, att in enumerate(attachments, 1):
                print(f"  {i}. Имя файла: {att.get('filename', 'unknown')}")
                print(f"     • Тип контента: {att.get('content_type', 'unknown')}")
                print(f"     • Размер: {att.get('size', 0)} байт")
                print(f"     • Content-Disposition: {att.get('content_disposition', 'N/A')}")
                
                # Ищем соответствующий хэш
                filename = att.get('filename', 'unknown')
                for hash_info in attachment_hashes:
                    if hash_info.get('filename') == filename:
                        print(f"     • SHA-256: {hash_info.get('sha256', 'N/A')}")
                        break
        else:
            print("  Вложения не найдены")
        
        # MULTIPART СТРУКТУРА (полная информация)
        multipart = parsed_mail.get('multipart', {})
        print(f"\n📦 MULTIPART СТРУКТУРА:")
        print("-" * 80)
        if multipart.get('is_multipart'):
            print(f"  • Это multipart: Да")
            print(f"  • Тип: {multipart.get('type', 'unknown')}")
            print(f"  • Content-Type: {multipart.get('content_type', 'N/A')}")
            parts = multipart.get('parts', [])
            print(f"  • Количество частей: {len(parts)}")
            if parts:
                print(f"\n  ДЕТАЛИ ЧАСТЕЙ:")
                for i, part in enumerate(parts, 1):
                    print(f"    Часть {i}:")
                    print(f"      • Content-Type: {part.get('content_type', 'unknown')}")
                    print(f"      • Размер: {part.get('size', 0)} байт")
        else:
            print(f"  • Это multipart: Нет")
            print(f"  • Тип: {multipart.get('type', 'single')}")
        
        print("\n" + "=" * 80)
        print("✅ ПАРСИНГ ПИСЬМА УСПЕШЕН")
        print("=" * 80)
        
    except Exception as e:
        print(f"\n❌ ОШИБКА ПРИ ПАРСИНГЕ ПИСЬМА:")
        print(f"   {type(e).__name__}: {str(e)}")
        import traceback
        traceback.print_exc()



In [26]:
parse_and_print_email_by_index(df, 4962)

ПАРСИНГ ПИСЬМА ПО ИНДЕКСУ 4962
Индекс: 4962
Метка (label): 0 (Легитимное)
Длина исходного текста: 7962 символов

--------------------------------------------------------------------------------
ИСХОДНОЕ ПИСЬМО (первые 500 символов):
--------------------------------------------------------------------------------
Message-ID: <11424477.1075853924812.JavaMail.evans@thyme>
Date: Thu, 7 Dec 2000 02:45:00 -0800 (PST)
From: wmontg6626@aol.com
To: awilliams@matrixservice.com, clott@matrixservice.com, ddavis@ect.enron.com, 
	smithflora@yahoo.com, ma4pamoore@aol.com, tatiandme@aol.com
Subject: Fwd: bring out the kleenex
Mime-Version: 1.0
Content-Type: text/plain; charset=ANSI_X3.4-1968
Content-Transfer-Encoding: 7bit
X-From: WMontg6626@aol.com
X-To: awilliams@matrixservice.com, clott@matrixservice.com, ddavis@ect
...

--------------------------------------------------------------------------------


2025-11-24 11:02:34,646 - email_parser - DEBUG - Email successfully parsed, found 6 headers
2025-11-24 11:02:34,646 - email_parser - DEBUG - parse_eml executed in 0.2339 sec
2025-11-24 11:02:34,646 - email_parser - DEBUG - Extracted 6 headers
2025-11-24 11:02:34,661 - email_parser - DEBUG - Extracted body: text=7274 chars, html=0 chars
2025-11-24 11:02:34,661 - email_parser - DEBUG - Extracted body: text=7274 chars, html=0 chars
2025-11-24 11:02:34,834 - email_parser - DEBUG - Extracted 0 unique URLs
2025-11-24 11:02:34,834 - email_parser - DEBUG - Extracted body: text=7274 chars, html=0 chars
2025-11-24 11:02:35,005 - email_parser - DEBUG - Extracted 0 unique URLs
2025-11-24 11:02:35,020 - email_parser - DEBUG - Extracted 6 headers
2025-11-24 11:02:35,020 - email_parser - DEBUG - Extracted domains: 2, IPs: 0, email domains: 2
2025-11-24 11:02:35,020 - email_parser - INFO - Email successfully parsed: headers=6, URLs=0, domains=2


РЕЗУЛЬТАТЫ ПАРСИНГА:
--------------------------------------------------------------------------------

📧 ВСЕ ЗАГОЛОВКИ (6):
--------------------------------------------------------------------------------
  • DATE: 2000-12-07 02:45:00-08:00
  • FROM: wmontg6626@aol.com
  • HEADER: {'to': ['awilliams@matrixservice.com, clott@matrixservice.com, ddavis@ect.enron.com, smithflora@yahoo.com, ma4pamoore@aol.com, tatiandme@aol.com'], 'x-from': ['WMontg6626@aol.com'], 'content-type': ['...
  • RECEIVED: None
  • SUBJECT: Fwd: bring out the kleenex
  • TO: awilliams@matrixservice.com

📝 ТЕЛО ПИСЬМА:
--------------------------------------------------------------------------------
  • Текст (text/plain): 7274 символов

  ПОЛНЫЙ ТЕКСТ:
  ----------------------------------------------------------------------------
  Return-path: <LawyerT@aol.com>
  From: LawyerT@aol.com
  Full-name: LawyerT
  Message-ID: <6.efee5a2.2760f364@aol.com>
  Date: Thu, 7 Dec 2000 09:06:28 EST
  Subject: Fwd: bring out the 

In [27]:
# Найти номер письма с указанным содержимым
import re

search_text = "From jose@monkey.org Fri Sep 27 15:46:37 2024"  
mask = df['email_content'].str.contains(re.escape(search_text), na=False)
matches = df[mask]

if not matches.empty:
    print("Номер(а) письма (индекса) с нужным содержимым:")
    print(matches.index.tolist())
else:
    print("Письмо с таким содержимым не найдено.")


Номер(а) письма (индекса) с нужным содержимым:
[1]


In [28]:
print(df.loc[4962, "email_content"])


Message-ID: <11424477.1075853924812.JavaMail.evans@thyme>
Date: Thu, 7 Dec 2000 02:45:00 -0800 (PST)
From: wmontg6626@aol.com
To: awilliams@matrixservice.com, clott@matrixservice.com, ddavis@ect.enron.com, 
	smithflora@yahoo.com, ma4pamoore@aol.com, tatiandme@aol.com
Subject: Fwd: bring out the kleenex
Mime-Version: 1.0
Content-Type: text/plain; charset=ANSI_X3.4-1968
Content-Transfer-Encoding: 7bit
X-From: WMontg6626@aol.com
X-To: awilliams@matrixservice.com, clott@matrixservice.com, ddavis@ect.enron.com, smithflora@yahoo.com, Ma4pamoore@aol.com, Tatiandme@aol.com
X-cc: 
X-bcc: 
X-Folder: \Dana_Davis_Dec2000\Notes Folders\All documents
X-Origin: Davis-D
X-FileName: ddavis2.nsf

Return-path: <LawyerT@aol.com>
From: LawyerT@aol.com
Full-name: LawyerT
Message-ID: <6.efee5a2.2760f364@aol.com>
Date: Thu, 7 Dec 2000 09:06:28 EST
Subject: Fwd: bring out the kleenex
To: Alicia_A_Wesley@comerica.com, AlmaHWesley@bankunited.com, 
clene62@hotmail.com, edskip@hal-pc.org, Faithdc98@aol.com, 
jacki

In [29]:
# Тестовое multipart письмо (multipart/alternative)
multipart_email = df.loc[4962, "email_content"]

# Тестовое письмо с вложениями (multipart/mixed)
mixed_email = df.loc[1, "email_content"]

# Simple письмо (НЕ multipart)
simple_email = """From: sender@example.com
To: recipient@example.com
Subject: Simple Email
Content-Type: text/plain

Simple text email.
"""

print("="*60)
print("ТЕСТ 1: multipart/alternative")
print("="*60)
result1 = parse_email(multipart_email)
mp1 = result1['multipart']
print(f"Is multipart: {mp1['is_multipart']}")
print(f"Type: {mp1['type']}")
print(f"Number of parts: {len(mp1['parts'])}")
for i, part in enumerate(mp1['parts'], 1):
    print(f"  Part {i}: {part['content_type']} ({part['size']} bytes)")
print(f"Body text: {len(result1['body']['text'])} chars")
print(f"Body HTML: {len(result1['body']['html'])} chars")

print("\n" + "="*60)
print("ТЕСТ 2: multipart/mixed (с вложением)")
print("="*60)
result2 = parse_email(mixed_email)
mp2 = result2['multipart']
print(f"Is multipart: {mp2['is_multipart']}")
print(f"Type: {mp2['type']}")
print(f"Number of parts: {len(mp2['parts'])}")
for i, part in enumerate(mp2['parts'], 1):
    print(f"  Part {i}: {part['content_type']} ({part['size']} bytes)")

print("\n" + "="*60)
print("ТЕСТ 3: простое письмо (не multipart)")
print("="*60)
result3 = parse_email(simple_email)
mp3 = result3['multipart']
print(f"Is multipart: {mp3['is_multipart']}")
print(f"Type: {mp3['type']}")

print("\n" + "="*60)
print("✓ ВСЕ ТЕСТЫ ЗАВЕРШЕНЫ")
print("="*60)










ТЕСТ 1: multipart/alternative


2025-11-24 11:02:35,518 - email_parser - DEBUG - Email successfully parsed, found 6 headers
2025-11-24 11:02:35,518 - email_parser - DEBUG - parse_eml executed in 0.2024 sec
2025-11-24 11:02:35,534 - email_parser - DEBUG - Extracted 6 headers
2025-11-24 11:02:35,534 - email_parser - DEBUG - Extracted body: text=7274 chars, html=0 chars
2025-11-24 11:02:35,534 - email_parser - DEBUG - Extracted body: text=7274 chars, html=0 chars
2025-11-24 11:02:35,712 - email_parser - DEBUG - Extracted 0 unique URLs
2025-11-24 11:02:35,712 - email_parser - DEBUG - Extracted body: text=7274 chars, html=0 chars
2025-11-24 11:02:35,863 - email_parser - DEBUG - Extracted 0 unique URLs
2025-11-24 11:02:35,863 - email_parser - DEBUG - Extracted 6 headers
2025-11-24 11:02:35,878 - email_parser - DEBUG - Extracted domains: 2, IPs: 0, email domains: 2
2025-11-24 11:02:35,878 - email_parser - INFO - Email successfully parsed: headers=6, URLs=0, domains=2
2025-11-24 11:02:36,080 - charset_normalizer - DEBUG - En

Is multipart: False
Type: single
Number of parts: 0
Body text: 7274 chars
Body HTML: 0 chars

ТЕСТ 2: multipart/mixed (с вложением)


2025-11-24 11:02:36,097 - email_parser - DEBUG - Email successfully parsed, found 11 headers
2025-11-24 11:02:36,121 - email_parser - DEBUG - parse_eml executed in 0.2430 sec
2025-11-24 11:02:36,121 - email_parser - DEBUG - Extracted 11 headers
2025-11-24 11:02:36,128 - email_parser - DEBUG - Extracted body: text=74 chars, html=4782 chars
2025-11-24 11:02:36,128 - email_parser - DEBUG - Extracted metadata for 1 attachments
2025-11-24 11:02:36,128 - email_parser - DEBUG - Computed 1 SHA-256 hashes for attachments
2025-11-24 11:02:36,128 - email_parser - DEBUG - Extracted body: text=74 chars, html=4782 chars
2025-11-24 11:02:36,144 - email_parser - DEBUG - Extracted 0 unique URLs
2025-11-24 11:02:36,144 - email_parser - DEBUG - Extracted body: text=74 chars, html=4782 chars
2025-11-24 11:02:36,160 - email_parser - DEBUG - Extracted 0 unique URLs
2025-11-24 11:02:36,160 - email_parser - DEBUG - Extracted 11 headers
2025-11-24 11:02:36,175 - email_parser - DEBUG - Extracted domains: 3, IPs

Is multipart: False
Type: single
Number of parts: 0

ТЕСТ 3: простое письмо (не multipart)
Is multipart: False
Type: single

✓ ВСЕ ТЕСТЫ ЗАВЕРШЕНЫ
